In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import bs4
import ssl
import urllib3

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
import pandas as pd
from langchain.docstore.document import Document

file_path = "./data/idiom_dict.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
lines = [line.strip() for line in lines]

docs = [Document(page_content=f"{line}") for line in lines]

In [3]:
print(len(docs))

22138


In [4]:
print(docs[0].page_content)

가슴이 뜨겁다 > Have a passionate heart


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

/var/tmp/ipykernel_63326/3629734522.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
/home/bun.2/.cache/pypoetry/virtualenvs/poetry-env-eDEwtiIl-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import os
faiss_index_path = "./embed_vector/multilingual-e5-large"
if os.path.exists(faiss_index_path):
    print("캐시된 벡터 로드...")
    vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
else:
    print("새로운 벡터를 생성하고 저장...")
    vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)
    vectorstore.save_local(faiss_index_path)

캐시된 벡터 로드...


In [ ]:
# ✅ FAISS 인덱스를 GPU로 변환 (공통 적용)
import faiss
try:
    # FAISS 인덱스를 GPU로 변환
    res = faiss.StandardGpuResources()
    gpu_index = faiss.index_cpu_to_gpu(res, 0, vectorstore.index)
    vectorstore.index = gpu_index
    print("GPU로 성공적으로 인덱스를 변환했습니다!")
    print(type(gpu_index))
except faiss.FaissException as e:
    print(f"FAISS 에러 발생: {e}")
    print("GPU 메모리가 부족하거나, 다른 이유로 변환에 실패했습니다.")

if isinstance(vectorstore.index, faiss.GpuIndex):  # GPU 인덱스이면
    print("🚀 FAISS 인덱스가 GPU에서 실행 중입니다!")
else:
    print("💻 FAISS 인덱스가 CPU에서 실행 중입니다!")

In [8]:
retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k':10})

In [9]:
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fab11317970> search_kwargs={'k': 10}


In [10]:
template = "용어집을 반영해서 한국어를 영어로 번역해줘. \
첫 번째 번역은 <translate_start1>으로 시작하고 <translate_end1>로 끝내줘. \
그 다음 틀린 부분이 있나 검토하고 두 번 째 번역은 <translate_start2>로 시작하고 <translate_end2>로 끝내줘. \
마지막으로 번역한 문장을 다시 한국어로 번역한 후에, 주어진 한국어({question})와 일치하는지 검토해. \
일치하면 두 번 째 번역과 동일하게 번역하고, 일치하지 않으면 한국어와 동일한 의미를 가진 영어로 다시 번역해줘. \
마지막 영어 번역은 <translate_start3>로 시작하고 <translate_end3>로 끝내줘. \
용어집에 유사한 표현이 있으면 해당 영어 표현을 꼭 참고해서 번역해줘. \
용어집 : {context}\n\n한국어 : {question}"

# template = "용어집을 반영해서 한국어를 영어로 번역해줘. \
# 번역은 <translate_start>로 시작하고 <translate_end>로 끝내줘. \
# 용어집에 유사한 표현이 있으면 해당 영어 표현을 꼭 참고해서 번역해줘. \
# 용어집 : {context}\n\n한국어 : {question}"
input_prompt = ChatPromptTemplate.from_template(template)

In [11]:
llm = ChatOllama(model='qwen:latest', temperature=0)
#llm = ChatOllama(model='llama3:70b', temperature=0)

/var/tmp/ipykernel_617727/2261560295.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model='qwen:latest', temperature=0)


In [12]:
output_parser = StrOutputParser()

In [13]:
def merge_info(docs,question):
    return {"context":'\n\n'.join(doc.page_content for doc in docs), "question":question}

In [14]:
def retrieve_and_merge(question):
    docs = retriever.invoke(question) #retriever로 문서 가져오고
    return merge_info(docs, question) #context와 question 포함한 딕셔너리 반환

In [15]:
#chain_level_3 = {'question' : RunnablePassthrough(), 'context':retriever | merge_info} | input_prompt | llm | output_parser
chain_level_3 = RunnablePassthrough() | retrieve_and_merge | input_prompt | llm | output_parser

import time

start = time.time()
print(chain_level_3.invoke('오늘만 넘기면 이제 목에 칼이 들어와도 넌 안전할 거야.'))
print("걸린 시간 : ",time.time()-start)

<translation_start1>
If you just get through today, no matter what happens from now on, you will be safe.
<translation_end1>

<translation_start2>
If you just make it through today, no matter what happens from now on, you'll be completely safe.
<translation_end2>

오늘만 넘기면 이제 목에 칼이 들어와도 넌 안전할 거야. -> If you just make it through today, no matter what happens from now on, you'll be completely safe.

<translation_start3>
If you just make it through today, no matter what happens from now on, you'll be completely safe.
<translation_end3> 

두 번째 번역과 세 번째 번역이 일치하며, 주어진 한국어 문장의 의미와 동일합니다.
걸린 시간 :  111.6017119884491


In [15]:
test_docs = retriever.invoke('오늘만 넘기면 이제 목에 칼이 들어와도 넌 안전할 거야.')
for d in test_docs:
    print(d)

page_content='호랑이에게 물려 가도 정신만 차리면 산다 > Stay calm in danger'
page_content='목에 칼이 들어와도 > No matter what happens'
page_content='내 턱에 칼이 들어와도 > No matter what happens'
page_content='털끝 하나 다치지 않다 > Stay completely safe'
page_content='호랑이굴에 들어가도 정신만 차리면 산다 > Stay calm even in danger'
page_content='칼을 뽑았으면 무라도 썰어야 한다 > If you start, see it through.'
page_content='범에게 물려 가도 정신만 차리면 산다 > Stay calm in crisis'
page_content='털 끝 하나 상하지 않다 > Be completely safe'
page_content='허리에 칼을 차다 > Be overly confident'
page_content='털끝 하나 다치지 않게 하다 > Keep completely safe'
page_content='칼끝이 서다 > Be on edge'
page_content='칼을 뽑았으면 무라도 잘라라 > Follow through once you've started'
page_content='얕은 내도 깊게 건너라 > Be cautious even in simple matters.'
page_content='호랑이가 물어가도 정신만 차리면 산다 > Stay calm in crisis'
page_content='호랑이에게 물려가도 정신만 차리면 산다 > Stay calm in a crisis'
page_content='호랑이 굴에 들어가도 정신만 차리면 산다 > Stay calm in crisis'
page_content='지키는 것이 곧 얻는 것이다 > Protecting what you have is valuable'
page_content='칼 끝에 주워 먹다 > B